# Data Preprocessing for Health Check Data

This notebook performs comprehensive data preprocessing on the health and nutrition datasets, including:
- Data cleaning
- Handling missing values
- Removing duplicates
- Data type conversions
- Normalization
- Data validation

## 1. Load Required Libraries and Data Files

First, we'll import the necessary libraries and load our datasets.

In [1]:
# Import required libraries
import pandas as pd
import numpy as np
from sklearn.preprocessing import StandardScaler, MinMaxScaler
import warnings
warnings.filterwarnings('ignore')

# Load the datasets
cnns_data = pd.read_csv('Comprehensive National Nutrition Survey (CNNS) 2016-18  Key Anthropometric Indicators by Sex.csv')
malnutrition_data = pd.read_csv('malnutrition_data (1).csv')
malnutrition_month = pd.read_csv('Malnutrition_month.csv')

# Display basic information about each dataset
print("CNNS Dataset Info:")
print(cnns_data.info())
print("\nMalnutrition Data Info:")
print(malnutrition_data.info())
print("\nMalnutrition Month Data Info:")
print(malnutrition_month.info())

CNNS Dataset Info:
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 62 entries, 0 to 61
Data columns (total 44 columns):
 #   Column                                                                                                                                Non-Null Count  Dtype  
---  ------                                                                                                                                --------------  -----  
 0   Rowid                                                                                                                                 62 non-null     int64  
 1   Country                                                                                                                               62 non-null     object 
 2   State lgd code                                                                                                                        62 non-null     int64  
 3   State                                                     

## 2. Handle Missing Values

Let's identify and handle missing values in each dataset.

In [2]:
# Check missing values in each dataset
print("Missing values in CNNS dataset:")
print(cnns_data.isnull().sum())
print("\nMissing values in Malnutrition dataset:")
print(malnutrition_data.isnull().sum())
print("\nMissing values in Malnutrition Month dataset:")
print(malnutrition_month.isnull().sum())

# Handle missing values in CNNS dataset
cnns_data_clean = cnns_data.copy()
# Fill numeric columns with median
numeric_columns = cnns_data_clean.select_dtypes(include=[np.number]).columns
cnns_data_clean[numeric_columns] = cnns_data_clean[numeric_columns].fillna(cnns_data_clean[numeric_columns].median())
# Fill categorical columns with mode
categorical_columns = cnns_data_clean.select_dtypes(exclude=[np.number]).columns
cnns_data_clean[categorical_columns] = cnns_data_clean[categorical_columns].fillna(cnns_data_clean[categorical_columns].mode().iloc[0])

# Handle missing values in Malnutrition dataset
malnutrition_data_clean = malnutrition_data.copy()
malnutrition_data_clean = malnutrition_data_clean.fillna(method='ffill').fillna(method='bfill')

# Handle missing values in Malnutrition Month dataset
malnutrition_month_clean = malnutrition_month.copy()
malnutrition_month_clean = malnutrition_month_clean.fillna(method='ffill').fillna(method='bfill')

# Verify missing values have been handled
print("\nVerifying missing values after cleaning:")
print("\nCNNS dataset missing values:")
print(cnns_data_clean.isnull().sum().sum())
print("\nMalnutrition dataset missing values:")
print(malnutrition_data_clean.isnull().sum().sum())
print("\nMalnutrition Month dataset missing values:")
print(malnutrition_month_clean.isnull().sum().sum())

Missing values in CNNS dataset:
Rowid                                                                                                                                   0
Country                                                                                                                                 0
State lgd code                                                                                                                          0
State                                                                                                                                   0
Yearcode                                                                                                                                0
Year                                                                                                                                    0
From year                                                                                                                               0
Ge

## 3. Remove Duplicates

Now let's check for and remove any duplicate records in our datasets.

In [3]:
# Check for duplicates in each dataset
print("Duplicate rows in CNNS dataset:", cnns_data_clean.duplicated().sum())
print("Duplicate rows in Malnutrition dataset:", malnutrition_data_clean.duplicated().sum())
print("Duplicate rows in Malnutrition Month dataset:", malnutrition_month_clean.duplicated().sum())

# Remove duplicates
cnns_data_clean = cnns_data_clean.drop_duplicates()
malnutrition_data_clean = malnutrition_data_clean.drop_duplicates()
malnutrition_month_clean = malnutrition_month_clean.drop_duplicates()

# Verify duplicates have been removed
print("\nAfter removing duplicates:")
print("Duplicate rows in CNNS dataset:", cnns_data_clean.duplicated().sum())
print("Duplicate rows in Malnutrition dataset:", malnutrition_data_clean.duplicated().sum())
print("Duplicate rows in Malnutrition Month dataset:", malnutrition_month_clean.duplicated().sum())

Duplicate rows in CNNS dataset: 0
Duplicate rows in Malnutrition dataset: 0
Duplicate rows in Malnutrition Month dataset: 4

After removing duplicates:
Duplicate rows in CNNS dataset: 0
Duplicate rows in Malnutrition dataset: 0
Duplicate rows in Malnutrition Month dataset: 0

After removing duplicates:
Duplicate rows in CNNS dataset: 0
Duplicate rows in Malnutrition dataset: 0
Duplicate rows in Malnutrition Month dataset: 0


## 4. Data Type Conversion

Let's check and convert data types where necessary.

In [4]:
# Check current data types
print("CNNS dataset data types:")
print(cnns_data_clean.dtypes)
print("\nMalnutrition dataset data types:")
print(malnutrition_data_clean.dtypes)
print("\nMalnutrition Month dataset data types:")
print(malnutrition_month_clean.dtypes)

# Convert numeric columns to appropriate types
def convert_numeric_columns(df):
    for col in df.select_dtypes(include=['float64']).columns:
        # If all values in the column are integers, convert to int
        if df[col].dropna().apply(lambda x: x.is_integer()).all():
            df[col] = df[col].astype('int64')
    return df

# Apply conversions to each dataset
cnns_data_clean = convert_numeric_columns(cnns_data_clean)
malnutrition_data_clean = convert_numeric_columns(malnutrition_data_clean)
malnutrition_month_clean = convert_numeric_columns(malnutrition_month_clean)

# Verify new data types
print("\nAfter type conversion:")
print("CNNS dataset data types:")
print(cnns_data_clean.dtypes)
print("\nMalnutrition dataset data types:")
print(malnutrition_data_clean.dtypes)
print("\nMalnutrition Month dataset data types:")
print(malnutrition_month_clean.dtypes)

CNNS dataset data types:
Rowid                                                                                                                                     int64
Country                                                                                                                                  object
State lgd code                                                                                                                            int64
State                                                                                                                                    object
Yearcode                                                                                                                                  int64
Year                                                                                                                                     object
From year                                                                                                      

## 5. Data Normalization

Now we'll normalize the numerical columns in our datasets.

In [5]:
# Function to normalize numerical columns
def normalize_numerical_columns(df):
    scaler = StandardScaler()
    numeric_columns = df.select_dtypes(include=[np.number]).columns
    if len(numeric_columns) > 0:
        df_normalized = df.copy()
        df_normalized[numeric_columns] = scaler.fit_transform(df[numeric_columns])
        return df_normalized
    return df

# Normalize each dataset
cnns_data_normalized = normalize_numerical_columns(cnns_data_clean)
malnutrition_data_normalized = normalize_numerical_columns(malnutrition_data_clean)
malnutrition_month_normalized = normalize_numerical_columns(malnutrition_month_clean)

# Display summary statistics of normalized data
print("Summary statistics of normalized CNNS data:")
print(cnns_data_normalized.describe())
print("\nSummary statistics of normalized Malnutrition data:")
print(malnutrition_data_normalized.describe())
print("\nSummary statistics of normalized Malnutrition Month data:")
print(malnutrition_month_normalized.describe())

Summary statistics of normalized CNNS data:
              Rowid  State lgd code  Yearcode  From year  \
count  6.200000e+01    6.200000e+01      62.0       62.0   
mean  -3.581365e-18    1.862310e-16       0.0        0.0   
std    1.008163e+00    1.008163e+00       0.0        0.0   
min   -1.704336e+00   -1.634146e+00       0.0        0.0   
25%   -8.521681e-01   -8.257653e-01       0.0        0.0   
50%    0.000000e+00   -6.637730e-02       0.0        0.0   
75%    8.521681e-01    8.399890e-01       0.0        0.0   
max    1.704336e+00    1.893334e+00       0.0        0.0   

       Stunted (height for age) children under the age of 5 years (%)  \
count                                       6.200000e+01                
mean                                        5.479488e-16                
std                                         1.008163e+00                
min                                        -2.177815e+00                
25%                                        -5.7849

## 6. Data Validation

Finally, let's perform some final checks on our preprocessed data.

In [6]:
# Final validation checks
def validate_dataset(df, name):
    print(f"\nValidation for {name}:")
    print(f"Number of rows: {len(df)}")
    print(f"Number of columns: {len(df.columns)}")
    print(f"Missing values: {df.isnull().sum().sum()}")
    print(f"Duplicate rows: {df.duplicated().sum()}")
    print("\nValue ranges for numerical columns:")
    print(df.describe())

# Perform validation for each dataset
validate_dataset(cnns_data_normalized, "CNNS Dataset")
validate_dataset(malnutrition_data_normalized, "Malnutrition Dataset")
validate_dataset(malnutrition_month_normalized, "Malnutrition Month Dataset")

# Save preprocessed datasets
cnns_data_normalized.to_csv('preprocessed_cnns_data.csv', index=False)
malnutrition_data_normalized.to_csv('preprocessed_malnutrition_data.csv', index=False)
malnutrition_month_normalized.to_csv('preprocessed_malnutrition_month.csv', index=False)

print("\nPreprocessed datasets have been saved to:")
print("- preprocessed_cnns_data.csv")
print("- preprocessed_malnutrition_data.csv")
print("- preprocessed_malnutrition_month.csv")


Validation for CNNS Dataset:
Number of rows: 62
Number of columns: 44
Missing values: 0
Duplicate rows: 0

Value ranges for numerical columns:
              Rowid  State lgd code  Yearcode  From year  \
count  6.200000e+01    6.200000e+01      62.0       62.0   
mean  -3.581365e-18    1.862310e-16       0.0        0.0   
std    1.008163e+00    1.008163e+00       0.0        0.0   
min   -1.704336e+00   -1.634146e+00       0.0        0.0   
25%   -8.521681e-01   -8.257653e-01       0.0        0.0   
50%    0.000000e+00   -6.637730e-02       0.0        0.0   
75%    8.521681e-01    8.399890e-01       0.0        0.0   
max    1.704336e+00    1.893334e+00       0.0        0.0   

       Stunted (height for age) children under the age of 5 years (%)  \
count                                       6.200000e+01                
mean                                        5.479488e-16                
std                                         1.008163e+00                
min                    